### Ground state solvers

In [1]:
###### Import Libraries

from qiskit import Aer
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
import time


In [2]:
###### Define molecule HF

molecule = Molecule(
    geometry=[["H", [0.0, 0.0, 0.0]], ["F", [0.0, 0.0, 0.9554]]], charge=0, multiplicity=1
)

print(molecule.geometry)

[['H', [0.0, 0.0, 0.0]], ['F', [0.0, 0.0, 0.9554]]]


In [3]:
###### Define the electronic structure driver

driver = ElectronicStructureMoleculeDriver(
    molecule, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF,
)

# define the electronic structure problem as a function of the driver
es_problem = ElectronicStructureProblem(driver) 

# build the second quantized Hamiltonian of the molecule
second_q_op = es_problem.second_q_ops()


# print the Hamiltonian, no. of spin orbitals or qubits, and the no. of paticles
print(second_q_op[0]) 
print(es_problem.num_spin_orbitals) 
print(es_problem.num_particles)

Fermionic Operator
register length=12, number terms=630
  (0.004795354445662637+0j) * ( +_0 +_1 -_2 -_5 )
+ (-0.031316822326622196+0j) * ( +_0 -_1 +_2 -_5 )
+ (0.026521467880959534+0j) * ( +_0 -_1 -_2 +_5 )
+ (0.07545120634168136+0j) * ( +_0 -_1 +_6 -_ ...
12
(5, 5)


In [4]:
###### Map Hamiltonian to Qubits
qubit_converter = QubitConverter(JordanWignerMapper()) 

### VQE solver

In [5]:
%%time
from qiskit.providers.aer import StatevectorSimulator
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_nature.algorithms import VQEUCCFactory


# Select the simulator to run the calculation
quantum_instance = QuantumInstance(backend=Aer.get_backend("aer_simulator_statevector"))

# define the VQE simulator
vqe_solver = VQEUCCFactory(quantum_instance)

CPU times: user 407 ms, sys: 51.8 ms, total: 459 ms
Wall time: 460 ms


### The calculation and results

In [6]:
%%time
from qiskit_nature.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(qubit_converter, vqe_solver) # do the calculation
res = calc.solve(es_problem) # save the result

print(res) # print the result

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -103.586665454064
  - computed part:      -103.586665454064
~ Nuclear repulsion energy (Hartree): 4.984922439062
> Total ground state energy (Hartree): -98.601743015002
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 10.000 S: 0.000 S^2: 0.000 M: -0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  16.24899905]
 
  0: 
  * Electronic dipole moment (a.u.): [0.00000011  0.00000031  16.68180831]
    - computed part:      [0.00000011  0.00000031  16.68180831]
  > Dipole moment (a.u.): [-0.00000011  -0.00000031  -0.43280926]  Total: 0.43280926
                 (debye): [-0.00000027  -0.00000078  -1.1000913]  Total: 1.1000913
 
CPU times: user 4min 49s, sys: 1min 26s, total: 6min 16s
Wall time: 4min 10s


### Just keeping

In [7]:
res.nuclear_repulsion_energy

4.984922439062173

In [8]:
res.electronic_energies

array([-103.58666545])

In [9]:
GSE = res.nuclear_repulsion_energy + res.electronic_energies 
GSE

array([-98.60174302])

In [10]:
print(res.computed_dipole_moment)

[(1.07174545586e-07, 3.06839972254e-07, 16.681808309641962)]


In [11]:
res.eigenstates

[array([ 2.61145619e-50+3.98202021e-51j, -8.09894474e-37-2.52727680e-37j,
        -5.65697958e-38-1.62394322e-38j, ...,
         1.24181404e-16+1.79138435e-16j, -6.03867360e-18-5.96539782e-17j,
         7.40502433e-17+5.00248193e-16j])]

In [12]:
res.num_particles

[9.999999999999904]

In [13]:
print(res.raw_result)

{   'aux_operator_eigenvalues': [   (9.999999999999904, 0.0),
                                    (5.92926292e-09, 0.0),
                                    (-0.0, 0.0),
                                    (1.07174545586e-07, 0.0),
                                    (3.06839972254e-07, 0.0),
                                    (16.681808309641962, 0.0)],
    'cost_function_evals': 449,
    'eigenstate': array([ 2.61145619e-50+3.98202021e-51j, -8.09894474e-37-2.52727680e-37j,
       -5.65697958e-38-1.62394322e-38j, ...,
        1.24181404e-16+1.79138435e-16j, -6.03867360e-18-5.96539782e-17j,
        7.40502433e-17+5.00248193e-16j]),
    'eigenvalue': (-103.58666545406436+0j),
    'optimal_parameters': {   ParameterVectorElement(t[0]): -9.582390494134314e-05,
                              ParameterVectorElement(t[1]): 0.0016693338389576905,
                              ParameterVectorElement(t[2]): 0.012183460404235185,
                              ParameterVectorElement(t[3]): -2.318